In [1]:
import cv2
import numpy as np

img = cv2.imread('C:/Users/Sahith/Desktop/IEEE Sheet Vision/ode_to_joy.jpg',0)
height, width = img.shape[:2]
print(height,width)
c1 = 540/height
c2 = 960/width

"""Resizing image"""
img_res = cv2.resize(img,(int(c2*width), int(c1*height)))#, interpolation = cv.INTER_CUBIC)


"""Otsu Binarisation"""
blur = cv2.GaussianBlur(img_res,(5,5),0)
if(c1 > 1.5 or c2 > 1.5):
    ret2,th2 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    print('ret2', ret2)
else:
    ret2,th2 = cv2.threshold(img_res,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    print('ret2', ret2)
    
"""Canny Edge Detection"""
edges = cv2.Canny(th2,50,150,apertureSize = 3)
#cv2.imshow('edges', edges)
"""Erosion - to get only the horizontal lines
    Subtract erosion from edges to remove the staff lines"""
kernel = np.ones((1,5), np.uint8)

erosion = cv2.erode(edges, kernel, iterations = 1)
#cv2.imshow('erosion', erosion)

result = cv2.subtract(edges, erosion)
#cv2.imshow('result', result)

"""To find the y-values of staff lines"""
sum_rows = np.sum(edges, axis=1)
max_sum = np.amax(sum_rows)

ind = np.where(sum_rows > 0.45 * max_sum)
print(ind)

new_ind = ind[0][::2] + 1  ## contains y-values of staff lines

ref_ind = new_ind[::5] ## Reference value for each set of 5 staff lines
ref_ind2 = new_ind[4::5]

bw = cv2.bitwise_not(th2)
horizontal = bw.copy()
vertical = bw.copy()

cv2.waitKey(0)
cv2.destroyAllWindows()

#Staff line thickness
from collections import Counter
ind = ind[0][:]
a = [x - ind[i-1] for i,x in enumerate(ind)][1:]
data = Counter(a)
b = data.most_common(1)
slt = b[0][0] #slt is staff line thickness
slt

#Removing the horizontal lines
kernel_v = np.ones((2*slt,1),dtype = np.uint8)
bw_erode1 = cv2.erode(vertical,kernel_v,iterations = 1)
bw_dilate1 = cv2.dilate(bw_erode1,kernel_v,iterations = 1)
#cv2.imshow('bw_dilate',bw_dilate1)
cv2.waitKey(0)
cv2.destroyAllWindows()

vertical_not = cv2.bitwise_not(bw_dilate1) #vertical
#cv2.imshow('not',vertical_not)
cv2.imshow('bw_dilate',bw_dilate1)
smooth = vertical_not.copy()
blur1 = cv2.filter2D(smooth,-1,(np.ones((2,1),np.float32)/2))
cv2.imwrite('C:/Users/Sahith/Desktop/IEEE Sheet Vision/staff_removal.jpg',blur1)
cv2.imshow('a',blur1)
cv2.waitKey(0)
cv2.destroyAllWindows()

1080 1920
ret2 160.0
(array([ 63,  65,  71,  73,  79,  81,  88,  90,  96,  98, 156, 158, 164,
       166, 172, 174, 181, 183, 189, 191, 351, 353, 360, 362, 368, 370,
       376, 378, 385, 387, 445, 447, 453, 455, 461, 463, 470, 472, 478,
       480], dtype=int64),)
